In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c44917e893cd26da060d42ad2716fb052b718ddad6572fce4f15b5368164154f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Missing').getOrCreate()
training = spark.read.csv('names.csv', header=True , inferSchema=True)

In [ ]:
training.show()

+-------+---+-------+
|   Name|Age| Salary|
+-------+---+-------+
| Saurin| 21|2000000|
|Devarya| 21| 400000|
|  Aryan| 25|3000000|
|  Manit| 18|5000000|
+-------+---+-------+



In [ ]:
training.columns

['Name', 'Age', 'Salary']

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
feature=VectorAssembler(inputCols=["Age"],outputCol="Indepedant feature")

In [ ]:
output=feature.transform(training)

In [ ]:
output.show()

+-------+---+-------+------------------+
|   Name|Age| Salary|Indepedant feature|
+-------+---+-------+------------------+
| Saurin| 21|2000000|            [21.0]|
|Devarya| 21| 400000|            [21.0]|
|  Aryan| 25|3000000|            [25.0]|
|  Manit| 18|5000000|            [18.0]|
+-------+---+-------+------------------+



In [ ]:
finaldata=output.select("Indepedant feature","Salary")

In [ ]:
finaldata.show()

+------------------+-------+
|Indepedant feature| Salary|
+------------------+-------+
|            [21.0]|2000000|
|            [21.0]| 400000|
|            [25.0]|3000000|
|            [18.0]|5000000|
+------------------+-------+



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
train_data,test_data=finaldata.randomSplit([0.67,0.33])
reg=LinearRegression(featuresCol='Indepedant feature',labelCol='Salary')
reg=reg.fit(train_data)


In [ ]:
reg.coefficients

DenseVector([-1000000.0])

In [ ]:
reg.intercept

22999999.99999936

In [ ]:
Pred_result=reg.evaluate(test_data)

In [ ]:
Pred_result.predictions.show()


+------------------+-------+-------------------+
|Indepedant feature| Salary|         prediction|
+------------------+-------+-------------------+
|            [21.0]| 400000| 2000000.0000000484|
|            [25.0]|3000000|-1999999.9999998212|
+------------------+-------+-------------------+



In [ ]:
Pred_result.meanAbsoluteError,Pred_result.meanSquaredError

(3299999.999999935, 13779999999999.184)